In [69]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import importlib as imp
import netCDF4 as nc
import os
import experiments
import tensorflow as tf
import keras

In [59]:
VARIABLE = "tos"
TIER = 1

PLOT_PRED_EXAMPLE = False
PLOT_AVG_ABS_DIFF = False

FULL_PERIOD_TREND = False #"Full-period trend (pattern correlation and RMSE)"
SUB_PERIOD_TREND = True #"Sub-period trend (pattern correlation and RMSE)"
SUB_YEAR_N_PERIOD = 30

#"Monthly and annual time series of global means and means in latitude bands (correlation and RMSE)"
#"Grid point, monthly & annual time series (global-mean correlation and MSE)"
#"Indices: ENSO, East-West Pacific SST gradient, NAO, SAM, Southern Ocean SST, Arctic Amplification factor (in tas)"

#IDEAS
#Ask Emily for pattern correlation


In [47]:
'''Plotting Functions'''
import matplotlib.colors

lower = plt.cm.RdBu_r(np.linspace(0,.49, 49))
white = plt.cm.RdBu_r(np.ones(2)*0.5)
upper = plt.cm.RdBu_r(np.linspace(0.51, 1, 49))
colors = np.vstack((lower, white, upper))
tmap = matplotlib.colors.LinearSegmentedColormap.from_list('terrain_map_white', colors)

def Line_Plot():
    print("plotting lines")

def Plot_Gobal_Map(plot_data, title, levels, colorbar, colorbar_title):
    plt.figure(dpi = (200), figsize = (6,4))
    ax=plt.axes(projection= ccrs.PlateCarree())
    data=plot_data
    data, lonsr = add_cyclic_point(data, coord=lons)
    cs=ax.contourf(lonsr, lats, data,
                transform = ccrs.PlateCarree(),extend='both', cmap = colorbar,
                levels = levels)

    ax.coastlines()
    cbar = plt.colorbar(cs,shrink=0.7,orientation='horizontal',label='Surface Air Temperature (K)', format='%.1f')#, pad=5)
    cbar.set_label(colorbar_title)
    plt.title(title)
    
    plt.show()

In [70]:
'''Load Data and Model'''

def load_npz(settings):
    npzdat = np.load(settings['npz_dir'] + settings['exp_name'] + '.npz')
    At, Ft, It, Av, Fv, Iv = npzdat['At'], npzdat['Ft'], npzdat['It'], npzdat['Av'], npzdat['Fv'], npzdat['Iv']
    return At, Ft, It, Av, Fv, Iv

Idata_file = nc.Dataset("/Users/charlotteconnolly/Documents/DATA/ForceSMIP/Evaluation/Amon/tas/tas_mon_1A.195001-202212.nc")
print(Idata_file)
Idata = Idata_file["tas"]
Itime = Idata_file["time"][:100]
lats = Idata_file['lat']
lons = Idata_file['lon']

Odata_file = nc.Dataset("/Users/charlotteconnolly/Documents/DATA/ForceSMIP/Evaluation/Amon/tas/tas_mon_1A.195001-202212.nc")
Truth = Odata_file["tas"][:100,:,:] #Truth
Truth_time = Odata_file["time"][:100]

from keras.models import load_model
model = load_model('/Users/charlotteconnolly/Documents/DATA/ForceSMIP/Models/tos_exp_0/tos_exp_00_model')
# model.summary()

Prediction = model.predict(Idata[:100, :, :])[:,:,:,0]


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    creation_date: Mon Jul 24 12:20:50 MDT 2023
    evaluation_id: 1A
    dimensions(sizes): time(876), lat(72), lon(144)
    variables(dimensions): float64 tas(time, lat, lon), float64 time(time), float64 lat(lat), float64 lon(lon)
    groups: 
4/4 [==============================] - 0s 1ms/step


In [80]:
if FULL_PERIOD_TREND:
    Truth_Trend = np.empty(shape = (72,144))
    Predicted_Trend = np.empty(shape = (72,144))
    for la in np.arange(0, lat_n, 1):
        for lo in np.arange(0, lon_n, 1):
            values = np.polyfit(Truth_time, Truth[:,la,lo], 1)
            Truth_Trend[la,lo] = values[0] * len(Truth_time)

            values = np.polyfit(Itime, Prediction[:,la,lo], 1)
            Predicted_Trend[la,lo] = values[0] * len(Itime)

    weighted_difference = (Predicted_Trend-Truth_Trend) * np.transpose(np.array([np.cos(np.deg2rad(lats)),]*144))
    RMSE = np.sum((weighted_difference)**2/len(Truth_time))
    print("The RMSE is: " +str(RMSE))
    Plot_Gobal_Map(Truth_Trend, "Spatial trend of Truth", None, tmap, "")
    Plot_Gobal_Map(Predicted_Trend, "Spatial trend of Prediction", None, tmap, "")
    Plot_Gobal_Map(Predicted_Trend-Truth_Trend, "Difference between truth and prediction", None, tmap, "")

if SUB_YEAR_N_PERIOD:
    sub_time = np.arange(0, len(Itime), SUB_YEAR_N_PERIOD)

    for s, sub in enumerate(sub_time[:-1]):
        Truth_Trend = np.empty(shape = (72,144))
        Predicted_Trend = np.empty(shape = (72,144))
        for la in np.arange(0, lat_n, 1):
            for lo in np.arange(0, lon_n, 1):
                values = np.polyfit(Truth_time[sub:sub_time[s+1]], Truth[sub:sub_time[s+1],la,lo], 1)
                Truth_Trend[la,lo] = values[0] * SUB_YEAR_N_PERIOD

                values = np.polyfit(Itime[sub:sub_time[s+1]], Prediction[sub:sub_time[s+1],la,lo], 1)
                Predicted_Trend[la,lo] = values[0] * SUB_YEAR_N_PERIOD

        weighted_difference = (Predicted_Trend-Truth_Trend) * np.transpose(np.array([np.cos(np.deg2rad(lats)),]*144))
        RMSE = np.sum((weighted_difference)**2/len(Truth_time))
        print("The RMSE is: " +str(RMSE))

The RMSE is: 0.0802942271702304
The RMSE is: 0.08670234017007064
The RMSE is: 0.09421339181488562


In [37]:
if PLOT_PRED_EXAMPLE:
    Plot_Gobal_Map(Prediction[0,:,:], "Example of a Prediction", None, tmap, "")

'''Average Absolute Difference'''
Difference = np.mean(np.abs(Prediction - Truth), axis = 0)

if PLOT_AVG_ABS_DIFF:
    Plot_Gobal_Map(Difference, "Average Absolute Difference between \nTruth and Prediction", None, tmap, "")